In [1]:
from libs import embeddings
from libs.graph import KnowledgeGraph

kg = KnowledgeGraph.from_dir("toy")
#E = embeddings.load("toy")

Triples: 100%|██████████| 316114/316114 [00:02<00:00, 123138.73it/s]


In [2]:
ename = "dbr:Anthony_Warde"
eid = kg.ent.to_id(ename)

kg.print_relations(ename)

dbr:Anthony_Warde
	 dbo:birthDate xsd:date
	 rdf:type
		 dbo:Agent
		 dbo:Person
	 dbo:activeYearsStartYear xsd:gYear
	 dbo:activeYearsEndYear xsd:gYear
	 dbo:imdbId <STRING>
	 dbo:birthPlace dbr:Pennsylvania
	 foaf:givenName <LABEL:en>
	 dbo:birthYear xsd:gYear
	 dbo:occupation dbr:Film
	 dbo:deathDate xsd:date
	 dbo:deathPlace
		 dbr:California
		 dbr:Hollywood
	 foaf:surname <LABEL:en>
	 foaf:gender <LABEL:en>
	 dbo:deathYear xsd:gYear


In [3]:
from importlib import reload
import libs.sampling.instance_sampling as samp
reload(samp)

samp.set_graph(kg)

instances, n = samp.sample_any(10)
print(f"Sampled {len(instances)} instances out of {n}")
print(*map(kg.ent.to_name, instances), sep="\n")

Sampled 10 instances out of 47694
dbr:Kolding
dbr:Gmina_Sławno,_Łódź_Voivodeship
dbr:Cadiz,_Kentucky
dbr:Jafarbay-ye_Jonubi_Rural_District
dbr:Colegio_Niño_Jesús_de_Praga
dbr:Lev_Kamenev
dbr:Andilanatoby
dbr:AC_Sparta_Praha_(women)
dbr:Whiting,_Vermont
dbr:The_Bahamas


In [4]:
# import libs.axiom.atomic as atoms
from libs.axiom import Concept, Existential
#reload(atoms)
reload(samp)
#Existential, Concept
samp.set_graph(kg)

Place = Concept(singleton="dbr:California")
a = Existential("dbo:deathPlace", Place)
b = Existential("dbo:deathYear", None)

instances, n = samp.sample_from(a & b, 10)
print(f"Sampled {len(instances)} instances out of {n} for axiom '{a & b}'")
print(*map(kg.ent.to_name, instances), sep="\n")

Sampled 7 instances out of 7 for axiom '∃dbo:deathPlace.{dbr:California}∧∃dbo:deathYear.⊤'
dbr:Ruth_Roland
dbr:A._A._Allen
dbr:Albert_R._Broccoli
dbr:Brad_Renfro
dbr:Anthony_Warde
dbr:Harvey_Lembeck
dbr:Isaac_Newton_Van_Nuys


In [5]:
kg.print_relations("dbr:California")

dbr:California
	 rdf:type
		 dbo:Location
		 dbo:Place
		 dbo:Region
		 dbo:PopulatedPlace
		 dbo:AdministrativeRegion


In [8]:
from collections import Counter

c = Counter()
for h, t in kg.iter_head_tails(kg.rel.to_id("rdf:type"), as_string=True):
    c[t] += 1

In [10]:
help(kg.sample_instances)

Help on method sample_instances in module libs.graph.knowledge_graph:

sample_instances(n, from_type='owl:Thing', except_type=None, type_rel='rdf:type', as_string=False, force_size=True, exclude_ids=None) method of libs.graph.knowledge_graph.KnowledgeGraph instance



In [12]:
classes = {cls for cls, count in c.most_common(n_classes + 1) if cls != "dbo:Place"}
classes

{'dbo:Agent',
 'dbo:Athlete',
 'dbo:Event',
 'dbo:Location',
 'dbo:Organisation',
 'dbo:Person',
 'dbo:PopulatedPlace',
 'dbo:Settlement',
 'dbo:SocietalEvent',
 'dbo:SportsEvent'}

In [16]:
raw_axioms = """
dbo:Athlete dbo:Person
dbo:Person dbo:Agent
dbo:Organisation dbo:Agent
dbo:SocietalEvent dbo:Event
dbo:SportsEvent dbo:Event
dbo:Settlement dbo:PopulatedPlace
dbo:PopulatedPlace dbo:Location
"""

axioms = [tuple(line.split()) for line in raw_axioms.split("\n")[1:-1]]
axioms

from libs.taxonomy import Taxonomy


T = Taxonomy.from_axioms(axioms, add_root="root")
T.print()

               ┌dbo:SocietalEvent
     ┌dbo:Event┤
     │         └dbo:SportsEvent
 root┤
     │         ┌dbo:Organisation
     ├dbo:Agent┤
     │         └dbo:Person┐
     │                    └dbo:Athlete
     └dbo:Location┐
                  └dbo:PopulatedPlace┐
                                     └dbo:Settlement


In [9]:
from sklearn.utils import shuffle
from libs.dataset import Dataset

n_classes = 10
n_entities = 200
classes = {cls for cls, count in c.most_common(n_classes + 1) if cls != "dbo:Place"}

used_indices = set()
indices = []
labels = []
name2cls, cls2name = dict(), dict()

for name in classes:
    cls = len(name2cls)
    name2cls[name] = cls
    cls2name[cls] = name
    
    for instance in kg.sample_instances(n_entities, from_type=name, exclude_ids=used_indices):
        used_indices.add(instance)
        indices.append(instance)
        labels.append(cls)
        
axioms = {
    "dbo:Athlete", "dbo:Person"
}

dbo:Location dbo:Agent dbo:Event dbo:SportsEvent dbo:Place dbo:PopulatedPlace dbo:Person dbo:SocietalEvent dbo:Organisation dbo:Settlement
